In [1]:
import numpy as np
import casadi as ca
import gym
import gym_custom_envs
import time as tt
import random
from gym import wrappers
from IPython.display import clear_output

In [2]:
class Model():
    def __init__(self):
        "Model Parameters"
        self.env = gym.make("Quadrotor2D-v0")
        self.m = self.env.m
        self.I = self.env.I
        self.gravity = self.env.gravity
        self.l = self.env.l
        self.r = self.l/2
        self.x_max = self.env.x_threshold
        self.y_max = self.env.y_threshold
        self.theta_max = self.env.O_threshold
        self.goal = self.env.goal
        if self.theta_max == 2*np.pi:
            self.theta_min = 0.
        elif self.theta_max == np.pi:
            self.theta_min = -np.pi
        
        "Model Hyper-Parameters"
        self.T = 2.
        self.u_max = abs(10*self.m*self.gravity)
        self.N = (self.T/self.env.dt).__int__()
        
    def modelDynamics(self, state, u):
        "Return the state_dot"
        x_ddot = -((u[0, 0] + u[1, 0])*ca.sin(state[2, 0]))/self.m
        y_ddot = (((u[0, 0] + u[1, 0])*ca.cos(state[2, 0])) - (self.m*self.gravity))/self.m
        theta_ddot = ((u[0, 0] - u[1, 0])*self.l/2)/self.I
        
        return x_ddot, y_ddot, theta_ddot

In [3]:
class Nlp(Model):
    def __init__(self, Model, start):
        "Initialise Optimization Variables"
        self.opti = ca.Opti()
        self.state = []
        self.u = []
        self.start = start
        self.goal = [Model.goal[0], Model.goal[1], Model.goal[2], 0., 0., 0.]
        
        for i in range(Model.N):
            self.state.append([self.opti.variable(3, 1), self.opti.variable(3, 1)])
            self.u.append(self.opti.variable(2, 1))
            
        self.ddq = []
        for i in range(Model.N):
            x_ddot, y_ddot, theta_ddot = Model.modelDynamics(self.state[i][0], self.u[i])
            self.ddq.append([x_ddot, y_ddot, theta_ddot])
        
        "Cost function"
        self.cost = self.getCost(self.u, Model.N, Model.T)
        self.opti.minimize(self.cost)
        
        "Constraints"
        self.constraints = self.getConstraints(Model, self.state, self.ddq, self.u)
        self.opti.subject_to(self.constraints)
        
        "Bounds"
        self.bounds = self.getBounds(Model, self.state, self.u)
        self.opti.subject_to(self.bounds)
        p_opts = {"expand":True}
        s_opts = {"max_iter": 1000}
        self.opti.solver("ipopt",p_opts,s_opts)
        
    def getCost(self, u, N, T):
        J = 0
        h = T/N
        for i in range(N-1):
            J += h*ca.sum1(u[i]**2 + u[i+1]**2)/2
        return J 
    
    def getConstraints(self, Model, state, ddq, u):
        ceq = []
        h = Model.T/Model.N
        
        "Collocation Constraints"
        for i in range(Model.N - 1):
            ceq.append((((state[i][0] + state[i+1][0])*h/2) - (state[i+1][1] - state[i][1]) == 0))
            for j in range(3):
                ceq.append((((state[i][1][j, 0] + state[i+1][1][j, 0])*h/2) - (ddq[i+1][j] - ddq[i][j]) == 0))
            
        "Boundary Constraints"
        ceq.append((state[0][0][0, 0] - self.start[0] == 0))
        ceq.append((state[0][0][1, 0] - self.start[1] == 0))
        ceq.append((state[0][0][2, 0] - self.start[2] == 0))
        ceq.append((state[0][1][0, 0] - self.start[3] == 0))
        ceq.append((state[0][1][1, 0] - self.start[4] == 0))
        ceq.append((state[0][1][2, 0] - self.start[5] == 0))
        ceq.append((state[-1][0][0, 0] - self.goal[0] == 0))
        ceq.append((state[-1][0][1, 0] - self.goal[1] == 0))
        ceq.append((state[-1][0][2, 0] - self.goal[2] == 0))
        ceq.append((state[-1][1][0, 0] - self.goal[3] == 0))
        ceq.append((state[-1][1][1, 0] - self.goal[4] == 0))
        ceq.append((state[-1][1][2, 0] - self.goal[5] == 0))
        ceq.append((u[-1][0, 0] - (Model.m*Model.gravity/2) == 0))
        ceq.append((u[-1][1, 0] - (Model.m*Model.gravity/2) == 0))
        return ceq
    
    def getBounds(self, Model, state, u):
        c = []
        f = 20
        for i in range(Model.N):
            q  = state[i][0]
            dq = state[i][1]
            tu  = u[i]
            c.extend([self.opti.bounded(-np.pi,q[0],np.pi),
                      self.opti.bounded(-np.pi,q[1],np.pi),
                      self.opti.bounded(-np.pi,q[2],np.pi),
                      self.opti.bounded(-f*np.pi,dq[0],f*np.pi),
                      self.opti.bounded(-f*np.pi,dq[1],f*np.pi),
                      self.opti.bounded(-f*np.pi,dq[2],f*np.pi),
                      self.opti.bounded(-Model.u_max,tu[0, 0],Model.u_max),
                      self.opti.bounded(-Model.u_max,tu[1, 0],Model.u_max)
                     ])
        return c

In [10]:
quad = Model()
s = quad.env.reset().reshape(1, 6)[0]
problem = Nlp(quad, s)
solution = problem.opti.solve()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    13580
Number of nonzeros in inequality constraint Jacobian.:     3200
Number of nonzeros in Lagrangian Hessian.............:     2000

Total number of variables............................:     3200
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2408
Total number of inequality constraints...............:     3200
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:     3200
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

  85r 1.9866981e-01 6.56e-01 8.13e+00  -5.6 2.81e+00    -  1.00e+00 3.16e-01f  1
  86r 1.9867088e-01 6.56e-01 7.68e+00  -5.6 1.36e+00    -  1.00e+00 8.74e-02f  1
  87r 1.9867822e-01 6.56e-01 2.14e+00  -5.6 1.21e+00    -  1.00e+00 7.22e-01f  1
  88r 1.9867936e-01 6.56e-01 9.66e-01  -5.6 2.99e-01    -  1.00e+00 5.35e-01f  1
  89r 1.9867991e-01 6.56e-01 8.98e-07  -5.6 1.37e-01    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90r 1.9868152e-01 6.56e-01 3.79e-03  -8.3 5.64e-02    -  9.93e-01 9.99e-01f  1
  91r 1.9868368e-01 6.56e-01 1.75e-01  -8.3 5.99e+01    -  7.23e-02 9.53e-03f  1
  92r 1.9868548e-01 6.56e-01 4.25e-01  -8.3 4.35e+01    -  1.15e-01 1.35e-02f  1
  93r 1.9868567e-01 6.56e-01 7.25e-01  -8.3 2.43e+01    -  1.43e-01 3.59e-03f  1
  94r 1.9868609e-01 6.56e-01 2.94e+00  -8.3 2.12e+01    -  9.33e-01 2.03e-02f  1
  95r 1.9868611e-01 6.56e-01 9.33e-01  -8.3 1.48e-02    -  1.00e+00 7.39e-01f  1

Number of Iterations....: 9

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Infeasible_Problem_Detected'

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
q = []; dq = []; u = []; pos = []; time = []

for j in range(3):
    tempq = [];tempdq = [];tempu = [];temp = []
    
    for i in range(quad.N):
        tempq.append(solution.value(problem.state[i][0][j]))    
        tempdq.append(solution.value(problem.state[i][1][j]))
        if j < 2:
            tempu.append(solution.value(problem.u[i][j]))
            
    q.append(tempq)
    dq.append(tempdq)
    pos.append(temp)   
    u.append(tempu)

time = np.arange(0.0, quad.T, quad.T/quad.N)

name = ['q','dq','u']

plt.subplot(311)
plt.title('Optimised Solution')
plt.plot(time, q[:][0],'r', time, q[:][1],'g',time ,q[:][2],'b')
plt.ylabel(name[0])

plt.subplot(312)
plt.plot(time, dq[:][0],'r',time, dq[:][1],'g',time, dq[:][2],'b')
plt.ylabel(name[1])

plt.subplot(313)
plt.plot(time, u[:][0],'g',time, u[:][1],'b')
plt.ylabel(name[2])

plt.suptitle('Quadrotor-2D')
plt.show()

In [ ]:
done = False
jj = 0
print(len(u[0][:]))
while jj < len(u[0][:]):    
#     a = [u_opt[jj].__float__(),u_opt[jj+1].__float__()]
#         print('Theta = ', round(np.rad2deg(s[0, 2]), 3))
#         print('u = ', a.T)
#         print("Reaction = ", round((a[0] + a[1])[0, 0] - m*g, 3))
#         print("Cost = ", round(np.dot(s, np.dot(S, s.T))[0, 0], 3))
    a = [u[0][jj], u[1][jj]]
    ns, r, done, _ = quad.env.step([a[0], a[1]])
    s = ns.reshape(1, 6)
    jj += 1
    quad.env.render()

    clear_output(wait=True)
    tt.sleep(0.001)
quad.env.close()